# Basic usage of LRP
This example will show you best practices for using LRP,
building on the basics shown in the *Getting started* section.

We start out by loading a small convolutional neural network:

In [1]:
using ExplainableAI
using Flux

model = Chain(
    Chain(
        Conv((3, 3), 3 => 8, relu; pad=1),
        Conv((3, 3), 8 => 8, relu; pad=1),
        MaxPool((2, 2)),
        Conv((3, 3), 8 => 16; pad=1),
        BatchNorm(16, relu),
        Conv((3, 3), 16 => 8, relu; pad=1),
        BatchNorm(8, relu),
    ),
    Chain(
        Flux.flatten,
        Dense(2048 => 512, relu),
        Dropout(0.5),
        Dense(512 => 100, softmax)
    ),
);

This model contains two chains: the convolutional layers and the fully connected layers.

## Model preparation
### Stripping the output softmax
When using LRP, it is recommended to explain output logits instead of probabilities.
This can be done by stripping the output softmax activation from the model
using the `strip_softmax` function:

In [2]:
model = strip_softmax(model)

Chain(
  Chain(
    Conv((3, 3), 3 => 8, relu, pad=1),  # 224 parameters
    Conv((3, 3), 8 => 8, relu, pad=1),  # 584 parameters
    MaxPool((2, 2)),
    Conv((3, 3), 8 => 16, pad=1),       # 1_168 parameters
    BatchNorm(16, relu),                # 32 parameters, plus 32
    Conv((3, 3), 16 => 8, relu, pad=1),  # 1_160 parameters
    BatchNorm(8, relu),                 # 16 parameters, plus 16
  ),
  Chain(
    Flux.flatten,
    Dense(2048 => 512, relu),           # 1_049_088 parameters
    Dropout(0.5),
    Dense(512 => 100),                  # 51_300 parameters
  ),
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.213 MiB.

If you don't remove the output softmax,
model checks will fail.

### Canonizing the model
LRP is not invariant to a model's implementation.
Applying the `GammaRule` to two linear layers in a row will yield different results
than first fusing the two layers into one linear layer and then applying the rule.
This fusing is called "canonization" and can be done using the `canonize` function:

In [3]:
model_canonized = canonize(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

After canonization, the first `BatchNorm` layer has been fused into the preceding `Conv` layer.
The second `BatchNorm` layer wasn't fused
since its preceding `Conv` layer has a ReLU activation function.

### Flattening the model
ExplainableAI.jl's LRP implementation supports nested Flux Chains and Parallel layers.
However, it is recommended to flatten the model before analyzing it.

LRP is implemented by first running a forward pass through the model,
keeping track of the intermediate activations, followed by a backward pass
that computes the relevances.

To keep the LRP implementation simple and maintainable,
ExplainableAI.jl does not pre-compute "nested" activations.
Instead, for every internal chain, a new forward pass is run to compute activations.

By "flattening" a model, this overhead can be avoided.
For this purpose, ExplainableAI.jl provides the function `flatten_model`:

In [4]:
model_flat = flatten_model(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, pad=1),         # 1_168 parameters
  BatchNorm(16, relu),                  # 32 parameters, plus 32
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.212 MiB.

This function is called by default when creating an LRP analyzer.
Note that we pass the unflattened model to the analyzer, but `analyzer.model` is flattened:

In [5]:
analyzer = LRP(model)
analyzer.model

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, pad=1),         # 1_168 parameters
  BatchNorm(16, relu),                  # 32 parameters, plus 32
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.212 MiB.

If this flattening is not desired, it can be disabled
by passing the keyword argument `flatten=false` to the `LRP` constructor.

## LRP rules
By default, the `LRP` constructor will assign the `ZeroRule` to all layers.

In [6]:
LRP(model)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => ZeroRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZeroRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, pad=1)       => ZeroRule(),
  BatchNorm(16, relu)                => ZeroRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZeroRule(),
  BatchNorm(8, relu)                 => ZeroRule(),
  Flux.flatten                       => ZeroRule(),
  Dense(2048 => 512, relu)           => ZeroRule(),
  Dropout(0.5)                       => ZeroRule(),
  Dense(512 => 100)                  => ZeroRule(),
)

This analyzer will return heatmaps that look identical to `InputTimesGradient`.

LRP's strength lies in assigning different rules to different layers,
based on their functionality in the neural network[^1].
ExplainableAI.jl implements many LRP rules out of the box,
but it is also possible to *implement custom rules*.

To assign different rules to different layers,
use one of the composites presets,
or create your own composite, as described in
*Assigning rules to layers*.

In [7]:
composite = EpsilonPlusFlat() # using composite preset EpsilonPlusFlat

Composite(
  GlobalTypeMap(  # all layers
    Flux.Conv               => ZPlusRule(),
    Flux.ConvTranspose      => ZPlusRule(),
    Flux.CrossCor           => ZPlusRule(),
    Flux.Dense              => EpsilonRule{Float32}(1.0f-6),
    typeof(NNlib.dropout)   => PassRule(),
    Flux.AlphaDropout       => PassRule(),
    Flux.Dropout            => PassRule(),
    Flux.BatchNorm          => PassRule(),
    typeof(Flux.flatten)    => PassRule(),
    typeof(MLUtils.flatten) => PassRule(),
    typeof(identity)        => PassRule(),
 ),
  FirstLayerTypeMap(  # first layer
    Flux.Conv          => FlatRule(),
    Flux.ConvTranspose => FlatRule(),
    Flux.CrossCor      => FlatRule(),
 ),
)

In [8]:
LRP(model, composite)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => FlatRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZPlusRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, pad=1)       => ZPlusRule(),
  BatchNorm(16, relu)                => PassRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZPlusRule(),
  BatchNorm(8, relu)                 => PassRule(),
  Flux.flatten                       => PassRule(),
  Dense(2048 => 512, relu)           => EpsilonRule{Float32}(1.0f-6),
  Dropout(0.5)                       => PassRule(),
  Dense(512 => 100)                  => EpsilonRule{Float32}(1.0f-6),
)

## Computing layerwise relevances
If you are interested in computing layerwise relevances,
call `analyze` with an LRP analyzer and the keyword argument
`layerwise_relevances=true`.

The layerwise relevances can be accessed in the `extras` field
of the returned `Explanation`:

In [9]:
input = rand(Float32, 32, 32, 3, 1) # dummy input for our convolutional neural network

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

([-6.186866 0.13927089 … 0.21902385 -4.1919928; 0.2089352 -3.6822412 … 21.524458 0.112474434; … ; -1.6981951 -0.32313746 … 7.23922 -2.4018502; -4.5555673 4.3152866 … 2.1715188 -0.95319235;;; 1.8259412 1.6724958 … -1.6384733 -2.8491237; 13.3870325 -2.0595298 … -0.06534412 -5.5749254; … ; -0.20498644 5.8345547 … 2.880249 -3.8290823; 1.260904 -2.459041 … 1.3053964 -1.150144;;; -2.3548925 2.039045 … -2.0076041 -5.8080525; 6.9608846 9.952976 … 14.896405 -3.1767855; … ; 0.19238053 9.278216 … 0.30768996 -0.4516734; -1.0047902 2.606806 … 5.439492 0.30238798;;;;], [-1.421106 10.7880125 … -0.0 -0.0; 1.1387973 -0.0 … -0.055267774 -0.0; … ; -0.5559021 -0.18083037 … -0.5264222 0.0; -0.0 0.0 … 0.0 0.0;;; 0.0 0.010384667 … 6.0941358 0.9529239; -0.0 9.544379 … 4.2414603 22.538452; … ; 0.0 -2.248032 … -1.1327746 11.947457; -0.0 -0.22577693 … -0.46530056 1.8915199;;; 0.8260882 5.6645093 … -0.21035577 0.00073682686; -1.673259 7.27547 … -3.3803518 -1.1566795; … ; 1.2372242 9.231191 … -5.499959 2.4014592; 

Note that the layerwise relevances are only kept for layers in the outermost `Chain` of the model.
When using our unflattened model, we only obtain three layerwise relevances,
one for each chain in the model and the output relevance:

In [10]:
analyzer = LRP(model; flatten=false) # use unflattened model

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

([1.5360225 -0.025504244 … -0.5069549 0.16323948; -1.3499297 7.236005 … -1.5457203 0.019549172; … ; -0.3658877 -0.08696673 … -3.8091505 -1.933056; -3.4234688 -1.2877731 … 0.61220294 0.30626568;;; -0.76808196 -1.3589213 … -0.41921565 0.30308163; -0.089706995 7.535221 … -1.5038174 0.28261274; … ; 0.39390564 -0.2622564 … 0.7568223 0.12954268; 1.1842526 1.099022 … 0.49424562 0.1126754;;; 1.275417 -0.023266578 … 0.40052876 0.35591733; -2.6093628 -0.8565462 … -0.7922556 -0.1814182; … ; -0.09016295 -3.1540296 … 2.9415653 -0.4358403; -0.30861974 -0.4893354 … 0.8077132 -0.5154748;;;;], [-0.00013999273 0.005355182 … 0.0070931963 -0.012414855; -0.0 -0.0003230527 … 0.0 -0.014425206; … ; 0.0 -0.0005850915 … 0.0 -0.0031752286; -0.0 0.0 … 0.0 0.0;;; 0.0 0.003122403 … -0.0 -0.004870654; -0.0 -0.0019768453 … -0.009498707 -0.002831374; … ; 0.0016649951 -0.001604005 … -0.012924797 0.0059397025; -4.265772f-5 0.0016018397 … -0.006185884 0.0008805082;;; 0.0 0.0 … -0.0 -0.0020848087; -0.0 0.020724382 … 0.019

## Performance tips
### Using LRP with a GPU
Like all other analyzers, LRP can be used on GPUs.
Follow the instructions on *GPU support*.

### Using LRP without a GPU
Using Julia's package extension mechanism,
ExplainableAI.jl's LRP implementation can optionally make use of
[Tullio.jl](https://github.com/mcabbott/Tullio.jl) and
[LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)
for faster LRP rules on dense layers.

This only requires loading the packages before loading ExplainableAI.jl:
```julia
using LoopVectorization, Tullio
using ExplainableAI
```

[^1]: G. Montavon et al., [Layer-Wise Relevance Propagation: An Overview](https://link.springer.com/chapter/10.1007/978-3-030-28954-6_10)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*